In [ ]:
#default_exp data.datasets.m3

# M3 dataset

> API details.
- https://forecasters.org/resources/time-series-data/m3-competition/
- https://www.sciencedirect.com/science/article/abs/pii/S0169207000000571?via%3Dihub
- https://robjhyndman.com/hyndsight/forecasting-competitions/

In [ ]:
#export
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd

from nixtla.data.datasets.utils import download_file, Info, TimeSeriesDataclass
from nixtla.data.tsdataset import TimeSeriesDataset

In [ ]:
#export
SOURCE_URL = 'https://forecasters.org/data/m3comp/M3C.xls'

M3 meta information

In [ ]:
#export
@dataclass
class Yearly:
    seasonality: int = 1
    horizon: int = 6
    freq: str = 'Y'
    sheet_name: str = 'M3Year'
    name: str = 'Yearly'
    n_ts: int = 645

@dataclass
class Quarterly:
    seasonality: int = 4
    horizon: int = 8
    freq: str = 'Q'
    sheet_name: str = 'M3Quart'
    name: str = 'Quarterly'
    n_ts: int = 756

@dataclass
class Monthly:
    seasonality: int = 12
    horizon: int = 18
    freq: str = 'M'
    sheet_name: str = 'M3Month'
    name: str = 'Monthly'
    n_ts: int = 1428

@dataclass
class Other:
    seasonality: int = 1
    horizon: int = 8
    freq: str = 'D'
    sheet_name: str = 'M3Other'
    name: str = 'Other'
    n_ts: int = 174

In [ ]:
#export
M3Info = Info(groups=('Yearly', 'Quarterly', 'Monthly', 'Other'),
              class_groups=(Yearly, Quarterly, Monthly, Other))

In [ ]:
#exporti
def _return_year(ts):
    year = ts.iloc[0]
    year = year if year != 0 else 1970

    return year

In [ ]:
#export
@dataclass
class M3(TimeSeriesDataclass):

    @staticmethod
    def load(directory: str,
             group: str) -> Tuple[pd.DataFrame, 
                                  Optional[pd.DataFrame], 
                                  Optional[pd.DataFrame]]:
        """
        Downloads and loads M3 data.

        Parameters
        ----------
        directory: str
            Directory where data will be downloaded.
        group: str
            Group name.
            Allowed groups: 'Yearly', 'Quarterly', 'Monthly', 'Other'.
            
        Notes
        -----
        [1] Returns train+test sets.
        [2] There are monthly time series without start year. 
            This time series will start with 1970.
        [3] Other time series have no start date.
            This time series will start with 1970.
        """
        path = Path(directory) / 'm3' / 'datasets'

        M3.download(directory)

        class_group = M3Info.get_group(group)

        df = pd.read_excel(path / 'M3C.xls', sheet_name=class_group.sheet_name)

        df = df.rename(columns={'Series': 'unique_id',
                                'Category': 'category',
                                'Starting Year': 'year',
                                'Starting Month': 'month'})

        df['unique_id'] = [class_group.name[0] + str(i + 1) for i in range(len(df))]
        S = df.filter(items=['unique_id', 'category'])

        id_vars = list(df.columns[:6])
        df = pd.melt(df, id_vars=id_vars, var_name='ds', value_name='y')
        df = df.dropna().sort_values(['unique_id', 'ds']).reset_index(drop=True)

        freq = pd.tseries.frequencies.to_offset(class_group.freq)

        if group == 'Other':
            df['year'] = 1970

        df['ds'] = df.groupby('unique_id')['year'] \
                     .transform(lambda df: pd.date_range(f'{_return_year(df)}-01-01',
                                                         periods=df.shape[0],
                                                         freq=freq))

        df = df.filter(items=['unique_id', 'ds', 'y'])
        
        return df, None, None

    @staticmethod
    def download(directory: Path) -> None:
        """Download M3 Dataset."""
        path = Path(directory) / 'm3' / 'datasets'
        if not path.exists():
            download_file(path, SOURCE_URL)

In [ ]:
for group, meta in M3Info:
    data, *_ = M3.load(directory='../data', group=group)
    unique_elements = data.groupby(['unique_id', 'ds']).size()
    unique_ts = data.groupby('unique_id').size()

    assert (unique_elements != 1).sum() == 0, f'Duplicated records found: {group}'
    assert unique_ts.shape[0] == meta.n_ts, f'Number of time series not match: {group}'